In [2]:
from sklearn import svm,datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
#定义网格参数 2*3=6个参数组合
params = {'kernel':('rbf','linear'),'C':[1,5,10]}
svr = svm.SVC()
clf = GridSearchCV(svr,param_grid=params)
# print clf
'''
这里的fit()函数会调用上面svr的fit()和predict()，然后取出params中的每一个参数组合进行评估
'''
clf.fit(iris.data,iris.target)
'''
6个参数组合都评估完成后，会把得分最高的模型参数放入best_estimator_
'''
print clf.best_estimator_

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [10]:
from scipy import stats
print stats.expon(scale=100)
print range(1,3)
from scipy.stats import randint as sp_randint
print sp_randint(2,11)

[1, 2]


In [8]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

#打印超参数搜索结果最好的函数
def reprot(results,n_top=3):
    for i in range(1,n_top+1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("model with rank : {0}".format(i))
            print "mean validation score : {0:.3f},std:{1:.3f}".format(
                results["mean_test_score"][candidate],
                results["std_test_score"][candidate]
            )
            print "parameters : {0}".format(results["params"][candidate])
            print ""

#获取数据：手写字符分类
digits = load_digits()
X,y = digits.data,digits.target
#构建一个分类器 参数是树的个数
clf = RandomForestClassifier(n_estimators=20)
#设置想要优化的超参数以及他们的取值分布 sp_randint(1,11)：从1到11之间均匀分布的整数中随机采样
params = {
    #树的最大深度
    "max_depth" : [3,None],
    #最大特征数
    "max_features" : sp_randint(1,11),
    #每个节点的最小样本划分数 这里必须从2开始 因为1个样本无法split
    "min_samples_split" : sp_randint(2,11),
    #每个叶子节点的最小样本划分数
    "min_samples_leaf" :  sp_randint(1,11),
    "bootstrap" : (True,False),
    #度量指标 gini指数和熵
    "criterion" : ("gini","entropy")
}
#开启超参数空间的随机搜索 20次里面每一次都会随机抽取一个参数组合进行K-Fold交叉验证
n_iter_search = 20
random_search = RandomizedSearchCV(clf,param_distributions=params,n_iter=n_iter_search)
start = time()
random_search.fit(X,y)
print "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings(候选参数组合)." % ((time()-start),n_iter_search)
reprot(random_search.cv_results_)

RandomizedSearchCV took 6.17 seconds for 20 candidates parameter settings.
model with rank : 1
mean validation score : 0.917,std:0.014
parameters : {'bootstrap': True, 'min_samples_leaf': 5, 'min_samples_split': 9, 'criterion': 'entropy', 'max_features': 9, 'max_depth': None}

model with rank : 2
mean validation score : 0.911,std:0.006
parameters : {'bootstrap': True, 'min_samples_leaf': 7, 'min_samples_split': 3, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}

model with rank : 3
mean validation score : 0.906,std:0.010
parameters : {'bootstrap': False, 'min_samples_leaf': 7, 'min_samples_split': 6, 'criterion': 'entropy', 'max_features': 3, 'max_depth': None}

